# Master RNN notebook

In [1]:
from __future__ import absolute_import, division, print_function 
#import os
#import re
#import sys
#import time

#sys.path.append(os.path.join('.', '..')) 
#import utils
#import utils_DL
import utils_s160159 as u_s

#import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

In [2]:
# https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/
# https://stats.stackexchange.com/questions/51296/how-do-you-calculate-precision-and-recall-for-multiclass-classification-using-co
def performance_measure(cm):
    TP = np.diag(cm)
    FP = np.sum(cm, axis=0) - np.diag(cm)
    FN = np.sum(cm,axis=1) - np.diag(cm)
    precision = u_s.safe_div(TP, TP + FP)
    recall = u_s.safe_div(TP, TP + FN)
    F1 = np.multiply(2, u_s.safe_div(np.multiply(precision, recall),
                                     np.add(precision, recall)))
    
    #
    return TP, FP, precision, recall, F1 

## Load data

In [3]:
data_dir = './../Data'
NUM_SUBJECTS = 6
NUM_CLASSES = 6
VAL_TRAIN_ID = NUM_SUBJECTS - 4
# Training Loop
MAX_EPOCHS = 1 # 50
BATCH_SIZE = 25 # 30 works on AWS 

In [4]:
cap = dict(np.load('./../Code/models/master/capture_dict.npz')['arr_0'].tolist())

In [5]:
cm = cap[1]['cm_val']

In [10]:
cap[1]

{'cm_test': array([[1810,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0]]),
 'cm_val': array([[7451,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0]]),
 'test_accuracy': 1.0,
 'test_loss': 0.0,
 'test_pred': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [6]:
## rnn
# test
rnn_test = np.array([[566,107,0,0,3,18],
 [122,125,11,0,4,79],
 [130,216,475,150,551,35],
 [9,2,15,31,184,0],
 [7,0,7,27,104,0],
 [128,278,10,0,1,317]])

# acc
rnn_acc = np.array([[113,52,3,1,1,25],
[25,64,5,0,1,22],
[16,48,221,99,234,5],
[6,9,24,31,125,0],
[4,2,4,44,268,0],
[32,113,23,0,0,172]])


cnn_test = np.array([[457,189,6,0,0,49],
[13,175,10,0,0,143],
[11,61,1225,69,34,157],
[9,1,18,87,126,0],
[5,0,0,13,127,0],
[1,52,4,0,0,677]])

# acc
cnn_acc = np.array([[126,66,0,1,1,19],
[6,48,4,1,0,58],
[2,32,440,80,2,67],
[3,0,20,97,75,0],
[1,0,0,11,309,1],
[0,8,3,0,0,329]])


In [4]:
rnn_acc

array([[113,  52,   3,   1,   1,  25],
       [ 25,  64,   5,   0,   1,  22],
       [ 16,  48, 221,  99, 234,   5],
       [  6,   9,  24,  31, 125,   0],
       [  4,   2,   4,  44, 268,   0],
       [ 32, 113,  23,   0,   0, 172]])

In [8]:
TP, FP, precision, recall, F1 = performance_measure(cnn_test)
print('--------------------------------------------')
print('Average for all classes')
print('Precision: %f' %(np.mean(precision)))
print('Recall:    %f' %(np.mean(recall)))
print('F1:        %f' %(np.mean(F1)))

--------------------------------------------
Average for all classes
Precision: 0.645756
Recall:    0.685182
F1:        0.640232


In [7]:
TP, FP, precision, recall, F1 = performance_measure(rnn_test)
print('--------------------------------------------')
print('Average for all classes')
print('Precision: %f' %(np.mean(precision)))
print('Recall:    %f' %(np.mean(recall)))
print('F1:        %f' %(np.mean(F1)))

--------------------------------------------
Average for all classes
Precision: 0.442481
Recall:    0.460826
F1:        0.376493


In [86]:
# http://www.dummies.com/education/science/biology/the-bootstrap-method-for-standard-errors-and-confidence-intervals/
def bootstrap_CI(values, n_sim = 100000, p = 0.05):
    x_hat_list = []
    # create X simulations
    for i in range(n_sim):
        # draw numbers from distribution
        x_hat = np.random.choice(values, 
                                 size=values.shape[0], 
                                 replace=True)
        # append mean value to list
        x_hat_list.append(np.mean(x_hat))
    # calculate the SD
    sd_hat = np.std(x_hat_list)
    mean_hat = np.mean(x_hat_list)
    # sort list
    x_hat_list = np.sort(x_hat_list)
    # remove lower and upper p/2 quantiles
    x_hat_list = x_hat_list[int(p/2*n_sim):int(n_sim-p/2*n_sim)]
    
    # calculate CI
    ci_l = mean_hat-sd_hat*(np.max(x_hat_list)-np.min(x_hat_list))
    ci_u = mean_hat+sd_hat*(np.max(x_hat_list)-np.min(x_hat_list))
    return(ci_l, mean_hat, ci_u)

tmp = np.random.normal(size=20, loc=100, scale=5)  
bootstrap_CI(tmp)


(92.631597657372424, 98.657546430616847, 104.68349520386127)